# Collecting morphologies from Neuromorpho.Org

This script calls the API provided by Neuromorpho.Org, and stores the image data in csv files for creation in the `1. Create the Rodent Basal Ganglia Graph.ipynb` notebook.

In [1]:
import requests, base64, os, json
import pandas as pd 
from io import BytesIO

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("..\Data/csvs/basal_ganglia/")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

csv_file_path = "file:///%s" % get_csv_path("experiments/cell_morphologies.csv")
df = pd.read_csv(csv_file_path)


class nmoInfo():
    def __init__(self, id, morph_id, href, img, archive):
        self.id = id
        self.morph_id = morph_id
        self.href = href
        self.img = img
        self.archive = archive
        
nmo_infos = []
        
for index, row in df.iterrows():
    nmoId = row["Neuromorpho_ID"];
    if(nmoId.startswith("NMO_00")):
        nmoId = nmoId[6:]
    elif(nmoId.startswith("NMO_0")):
        nmoId = nmoId[5:]
    else:
        nmoId = nmoId[4:]
    
    res = requests.get("http://neuromorpho.org/api/neuron/id/" + nmoId).content
    data = json.loads(res)
    
    try:
        picture_url = data["png_url"]
        img_data = requests.get(picture_url).content
        buffered = BytesIO(img_data)
        img_base64 = base64.b64encode(buffered.getvalue())

        href = "http://neuromorpho.org/neuron_info.jsp?neuron_name=" + data["neuron_name"]
        nmo_infos.append([row["ID"], nmoId, href, img_base64, data["archive"], data["reference_doi"]])

        if(nmoId == "68266"):
            print(img_base64)
    except:
        print("FAILED for", nmoId, row["Morphology_name"])
          

nmo_df = pd.DataFrame(nmo_infos, columns = ["id", "neuromorphoId", "href", "base64", "archive", "original_paper_doi"]) 
nmo_df.to_csv("..\Data/csvs/neuromorpho/neuromorpho.csv", index=1)

8371
830
8369
8368
8367
8366
8365
8364
8363
8362
8361
8360
8359
8358
8357
8356
8355
8354
8353
8352
8351
8350
8349
8348
8347
8346
8345
8344
8343
8342
8341
8340
8339
8338
8337
4518
4519
4520
4521
4522
4523
860
6825
68266
b'...'
9580


9579
9578
9577
9576
9575
